# 🧌 Monstrinho 3.5

O sol amanhece no reino de Lumi e um novo monstro aparece para incomodar. Vamos lá então. Para derrotar esse monstro, vamos utilizar 3 novas funções de ativação na rede neural. Realizaremos isso através dos novos conceitos aprendidos na aula de MLP (Multi-Layer Perceptron) onde montamos um modelo do zero com Python puro.

A seguir, estão as 4 classes (`Valor`, `Neurônio`, `Camada` e `MLP`) já criadas na aula de MLP. As mudanças que existem foram feitas por mim para melhorar a legibilidade e a organização do código.

## 🔢 Valor

Esta é a classe Valor, que representa um valor numérico com suporte a diferenciação automática. Ela possui métodos para operações matemáticas, como adição, subtração, multiplicação e exponenciação, além de métodos para calcular a função sigmoide e backpropagation. É notório lembrar que seja possível realizar os cálculos mesmo com ordem invertida, onde o objeto Valor pode ser o primeiro ou segundo operando.

In [1]:
import math

class Valor:
    """
    Classe que representa um valor numérico com suporte a diferenciação automática.
    
    Cada instância armazena:
      - data: o valor numérico.
      - progenitor: tupla com os valores dos quais este foi derivado.
      - operador_mae: string representando a operação que gerou o valor.
      - grad: gradiente (inicialmente zero) usado no backpropagation.
    """
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        """
        Inicializa uma instância de Valor.
        
        Args:
            data (float): o valor numérico.
            progenitor (tuple): valores anteriores que contribuíram para este.
            operador_mae (str): operação que gerou o valor.
            rotulo (str): rótulo opcional para identificação.
        """
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo
        self.grad = 0

    def __repr__(self):
        """Retorna uma representação string simplificada do objeto."""
        return f"Valor(data={self.data})"

    def __add__(self, outro_valor):
        """
        Sobrecarga do operador de adição.
        
        Realiza a operação: self + outro_valor.
        
        Args:
            outro_valor (Valor ou número): valor a ser somado.
        
        Returns:
            Valor: novo objeto representando a soma.
        """
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data + outro_valor.data
        operador_mae = "+"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_adicao():
            self.grad += resultado.grad
            outro_valor.grad += resultado.grad
            
        resultado.propagar = propagar_adicao
        
        return resultado

    def __mul__(self, outro_valor):
        """
        Sobrecarga do operador de multiplicação.
        
        Realiza a operação: self * outro_valor.
        
        Args:
            outro_valor (Valor ou número): valor a ser multiplicado.
        
        Returns:
            Valor: novo objeto representando o produto.
        """
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data * outro_valor.data
        operador_mae = "*"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_multiplicacao():
            self.grad += resultado.grad * outro_valor.data
            outro_valor.grad += resultado.grad * self.data
            
        resultado.propagar = propagar_multiplicacao
        
        return resultado

    def exp(self):
        """
        Calcula a exponencial do valor.
        
        Realiza a operação: exp(self).
        
        Returns:
            Valor: novo objeto representando a exponencial.
        """
        progenitor = (self, )
        data = math.exp(self.data)
        operador_mae = "exp"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_exp():
            self.grad += resultado.grad * data 
        
        resultado.propagar = propagar_exp
        
        return resultado

    def __pow__(self, expoente):
        """
        Sobrecarga do operador de exponenciação.
        
        Realiza a operação: self ** expoente.
        
        Args:
            expoente (int ou float): expoente da operação.
        
        Returns:
            Valor: novo objeto representando a exponenciação.
        """
        assert isinstance(expoente, (int, float)), "Expoente deve ser um número."
        progenitor = (self, )
        data = self.data ** expoente
        operador_mae = f"**{expoente}"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_pow():
            self.grad += resultado.grad * (expoente * self.data ** (expoente - 1))
        
        resultado.propagar = propagar_pow
        
        return resultado

    def __truediv__(self, outro_valor):
        """
        Sobrecarga do operador de divisão.
        
        Realiza a operação: self / outro_valor.
        
        Args:
            outro_valor (Valor ou número): divisor.
        
        Returns:
            Valor: novo objeto representando a divisão.
        """
        return self * outro_valor ** (-1)

    def __neg__(self):
        """
        Sobrecarga do operador de negação.
        
        Realiza a operação: -self.
        
        Returns:
            Valor: novo objeto representando o valor negativo.
        """
        return self * -1

    def __sub__(self, outro_valor):
        """
        Sobrecarga do operador de subtração.
        
        Realiza a operação: self - outro_valor.
        
        Args:
            outro_valor (Valor ou número): valor a ser subtraído.
        
        Returns:
            Valor: novo objeto representando a subtração.
        """
        return self + (-outro_valor)

    def __radd__(self, outro_valor):
        """
        Sobrecarga do operador de adição reversa.
        
        Permite operações onde Valor está à direita: outro_valor + self.
        
        Args:
            outro_valor (Valor ou número): valor a ser somado.
        
        Returns:
            Valor: resultado da adição.
        """
        return self + outro_valor

    def __rmul__(self, outro_valor):
        """
        Sobrecarga do operador de multiplicação reversa.
        
        Permite operações onde Valor está à direita: outro_valor * self.
        
        Args:
            outro_valor (Valor ou número): valor a ser multiplicado.
        
        Returns:
            Valor: resultado da multiplicação.
        """
        return self * outro_valor

    def sig(self):
        """
        Calcula a função sigmoide.
        
        Realiza a operação: exp(self) / (exp(self) + 1).
        
        Returns:
            Valor: novo objeto representando o resultado da sigmoide.
        """
        return self.exp() / (self.exp() + 1)

    def propagar(self):
        """
        Função de propagação (backpropagation) do gradiente.
        
        Este método deve ser sobrescrito pelas operações específicas.
        """
        pass

    def propagar_tudo(self):
        """
        Executa o backpropagation através de todos os nós (valores) conectados.
        
        Atribui gradiente 1 ao vértice folha e propaga recursivamente utilizando uma ordem topológica dos nós.
        """
        self.grad = 1
        ordem_topologica = []
        visitados = set()

        def constroi_ordem_topologica(v):
            if v not in visitados:
                visitados.add(v)
                for progenitor in v.progenitor:
                    constroi_ordem_topologica(progenitor)
                ordem_topologica.append(v)

        constroi_ordem_topologica(self)

        for vertice in reversed(ordem_topologica):
            vertice.propagar()


## 🤓 Neurônio

A classe Neurônio representa um neurônio em uma rede neural. Ela possui pesos e um viés, que são inicializados aleatoriamente. O neurônio calcula a saída usando a função sigmoide e realiza o backpropagation para atualizar os pesos e o viés com base no erro da previsão.

In [2]:
import random


class Neuronio:
    """
    Representa um neurônio simples com pesos e viés para uso em uma rede neural.

    Este neurônio utiliza a classe Valor para armazenar seus parâmetros e realizar a
    diferenciação automática durante o treinamento.
    """

    def __init__(self, num_dados_entrada, ativacao="sigmoid"):
        """
        Inicializa um neurônio com pesos e viés aleatórios.
        
        Args:
            num_dados_entrada (int): número de entradas do neurônio.
            ativacao (str): função de ativação a ser utilizada. Opções: "sigmoid", "relu", "tanh" e "leaky_relu".
        """
        self.vies = Valor(random.uniform(-1, 1))

        self.pesos = []
        for i in range(num_dados_entrada):
            self.pesos.append(Valor(random.uniform(-1, 1)))

        if ativacao == "sigmoid":
            self.ativacao = lambda x: x.sig()
        elif ativacao == "relu":
            self.ativacao = lambda x: x.relu()
        elif ativacao == "tanh":
            self.ativacao = lambda x: x.tanh()
        elif ativacao == "leaky_relu":
            self.ativacao = lambda x: x.leaky_relu()
        else:
            raise ValueError(f"Função de ativação '{ativacao}' não suportada.")

    def __call__(self, x):
        """
        Realiza a passagem forward do neurônio.

        Calcula a soma ponderada das entradas e aplica a função sigmoide para
        determinar a saída do neurônio.

        Args:
            x (list[Valor]): lista de objetos Valor representando as entradas.

        Returns:
            Valor: objeto Valor representando a saída do neurônio.
        """
        assert len(x) == len(
            self.pesos
        ), "O número de entradas deve ser igual ao número de pesos."

        soma = 0
        for info_entrada, peso_interno in zip(x, self.pesos):
            soma += info_entrada * peso_interno

        soma += self.vies

        return self.ativacao(soma)

    def parametros(self):
        """
        Retorna uma lista com os parâmetros do neurônio (pesos e viés).

        Returns:
            list[Valor]: lista contendo os pesos e o viés.
        """
        return self.pesos + [self.vies]

Uma pequena alteração ocorreu na classe `Neuronio` para permitir a utilização de funções de ativação diferentes. Para isso, foi adicionado um parâmetro `ativacao` no construtor da classe. Esse parâmetro pode ser uma função de ativação como `sigmoide`, `relu`, `tanh` ou `leaky_relu`. O método `ativar` agora utiliza essa função de ativação para calcular a saída do neurônio.

## 🎂 Camada

A classe Camada representa uma camada de neurônios em uma rede neural. Ela possui um número específico de neurônios e é responsável por calcular a saída da camada com base nas entradas recebidas. Realiza o forward pass e agrega os parametros de cada neurônio.

In [3]:
class Camada:
    """
    Representa uma camada em uma rede neural composta por múltiplos neurônios.

    Cada camada gerencia um conjunto de neurônios, realizando a passagem forward
    e agregando os parâmetros (pesos e viés) de cada neurônio.
    """

    def __init__(self, num_neuronios, num_dados_entrada, ativacao="sigmoid"):
        """
        Inicializa a camada com um número específico de neurônios, cada um com
        um determinado número de entradas.

        Args:
            num_neuronios (int): número de neurônios na camada.
            num_dados_entrada (int): número de entradas para cada neurônio.
        """
        self.neuronios = []
        for _ in range(num_neuronios):
            neuronio = Neuronio(num_dados_entrada, ativacao=ativacao)
            self.neuronios.append(neuronio)

    def __call__(self, x):
        """
        Realiza a passagem forward na camada.

        Aplica cada neurônio da camada à mesma entrada e retorna os dados de saída.

        Args:
            x (list[Valor]): lista de objetos Valor representando as entradas da camada.

        Returns:
            Valor ou list[Valor]: saída de um único neurônio se houver apenas um,
            ou lista com as saídas de todos os neurônios.
        """
        dados_de_saida = []
        for neuronio in self.neuronios:
            informacao = neuronio(x)
            dados_de_saida.append(informacao)

        if len(dados_de_saida) == 1:
            return dados_de_saida[0]
        else:
            return dados_de_saida

    def parametros(self):
        """
        Agrega e retorna todos os parâmetros (pesos e viés) de cada neurônio da camada.

        Returns:
            list[Valor]: lista contendo todos os parâmetros dos neurônios da camada.
        """
        params = []
        for neuronio in self.neuronios:
            params_neuronio = neuronio.parametros()
            params.extend(params_neuronio)

        return params

A classe `Camada` também foi alterada para permitir a utilização de funções de ativação diferentes. O construtor agora aceita um parâmetro `ativacao` que pode ser uma função de ativação como `sigmoide`, `relu`, `tanh` ou `leaky_relu`. O método `ativar` utiliza essa função de ativação para calcular a saída da camada.

## 🧠 MLP

Por fim, nossa última classe da nossa rede neural é a `MLP` (Multi-Layer Perceptron). Ela representa uma rede neural com múltiplas camadas. A `MLP` organiza as camadas da rede, permitindo a passagem forward dos dados e a agregação dos parâmetros de todas as camadas.

In [4]:
class MLP:
    """
    Representa uma rede neural do tipo MLP (Multi-Layer Perceptron).

    Essa classe organiza as camadas da rede, permitindo a passagem forward dos dados e
    a agregação dos parâmetros (pesos e viés) de todas as camadas.
    """

    def __init__(self, num_dados_entrada, num_neuronios_por_camada, ativacao="sigmoid"):
        """
        Inicializa a MLP com um número definido de entradas e uma lista que especifica
        o número de neurônios em cada camada.

        Args:
            num_dados_entrada (int): número de entradas da rede.
            num_neuronios_por_camada (list[int]): lista com o número de neurônios para cada camada.
        """
        percurso = [num_dados_entrada] + num_neuronios_por_camada

        camadas = []
        for i in range(len(num_neuronios_por_camada)):
            camada = Camada(num_neuronios_por_camada[i], percurso[i], ativacao=ativacao)
            camadas.append(camada)

        self.camadas = camadas

    def __call__(self, x):
        """
        Realiza a passagem forward pela rede.

        Cada camada processa a entrada e o resultado é passado para a próxima camada.

        Args:
            x (list[Valor] ou Valor): dados de entrada para a rede.

        Returns:
            Valor ou list[Valor]: saída final da rede após a passagem por todas as camadas.
        """
        for camada in self.camadas:
            x = camada(x)
        return x

    def parametros(self):
        """
        Agrega e retorna todos os parâmetros (pesos e viés) de todas as camadas da rede.

        Returns:
            list[Valor]: lista contendo os parâmetros de cada camada.
        """
        params = []
        for camada in self.camadas:
            parametros_camada = camada.parametros()
            params.extend(parametros_camada)

        return params

Por fim, a classe `MLP` também foi alterada para permitir a utilização de funções de ativação diferentes. O construtor agora aceita um parâmetro `ativacao` que pode ser uma função de ativação como `sigmoide`, `relu`, `tanh` ou `leaky_relu`. O método `ativar` utiliza essa função de ativação para calcular a saída da rede neural.

## 🪝 Funções de Ativação

Show! Agora que definimos todas as classes que usaremos para essa atividade, vamos abordar as 3 novas funções de ativação que vamos usar para treinar nossa rede neural. [1]

### 0️⃣ Sigmoide

Antes de mostrarmos as novas funções de ativação, vamos relembrar a função sigmoide. A função sigmoide é uma função matemática que transforma um valor real em um valor entre 0 e 1. Ela é definida pela fórmula:

$$
f(x) = \frac{1}{1 + e^{-x}}
$$

Sua principal característica é que ela "achata" os valores extremos, ou seja, valores muito grandes ou muito pequenos são aproximados para 1 ou 0, respectivamente. Isso pode ser útil em redes neurais, pois ajuda a evitar problemas de saturação e facilita o treinamento.

<center>
<img src="images/Sigmoid.jpg" alt="ReLU Function" width="300"/>
<p>Legenda 1: Esta é a função Sigmoide. Ela transforma valores reais em valores entre 0 e 1.</p>
</center>

A função sigmoide é amplamente utilizada em redes neurais, especialmente em problemas de classificação binária. Ela é uma função suave e diferenciável, o que a torna adequada para otimização usando algoritmos de aprendizado de máquina.

A função sigmoide já foi implementada na classe `Valor` e é utilizada como função de ativação padrão nos neurônios. Vamos citar a função sigmoide novamente aqui para relembrar como ela é implementada: 

In [5]:
def sig(self):
    """
    Calcula a função sigmoide.

    Realiza a operação: exp(self) / (exp(self) + 1).

    Returns:
        Valor: novo objeto representando o resultado da sigmoide.
    """
    return self.exp() / (self.exp() + 1)

> Você pode notar que existe uma pequena diferença entre a fórmula utilizada na função (definida pelo professor) e pela apresentada acima. Mas, é a mesma coisa, são equivalentes. Aqui está a explicação matemática para isso:
> $$
> f(x) = \frac{1}{1 + e^{-x}} \times \frac{e^{x}}{e^{x}} = \frac{e^{x}}{e^{x} + 1}
> $$

### 1️⃣ ReLU (Rectified Linear Unit)

A ReLU é uma das funções de ativação mais populares para redes neurais artificiais, e encontra aplicação em visão computacional e reconhecimento de fala usando redes neurais profundas e neurociência computacional. [2]

$$
\text{ReLU}(x) = \max(0, x)
$$

Uma das principais características da função de ativação ReLU são:

- Simples e eficiente
- Resolve parcialmente o problema do **desvanecimento do gradiente**.
- Pode "morrer" para entradas negativas (gradiente zero).

Quando dizemos que a ReLU é simples e eficiente, nos referimos ao fato de que ela é computacionalmente barata, pois envolve apenas uma comparação e não requer operações exponenciais ou trigonométricas. Isso a torna adequada para redes neurais profundas, onde a eficiência computacional é crucial. Já o problema do desvanecimento do gradiente ocorre quando os gradientes se tornam muito pequenos durante o treinamento, dificultando a atualização dos pesos. A ReLU ajuda a mitigar esse problema, permitindo que os gradientes fluam mais facilmente através das camadas da rede. No entanto, a ReLU pode "morrer" para entradas negativas, resultando em neurônios que não se ativam e não contribuem para o aprendizado. Isso pode ocorrer quando os pesos são atualizados de forma que a saída do neurônio permaneça negativa.

<center>
<img src="images/ReLU.jpg" alt="ReLU Function" width="300"/>
<p>Legenda 2: Esta é a função ReLU. Ela é linear para valores positivos e zero para valores negativos.</p>
</center>

Comparando com a função de ativação sigmoide que mapeia para o intervalo (0, 1), o que pode gerar gradientes pequenos em extremos. Mas, a ReLU já é mais eficiente e não satura no lado positivo, o que significa que os gradientes permanecem grandes e não se tornam pequenos, permitindo que a rede aprenda mais rapidamente.

Vamos aproveitar, e já criar a função em Python para a ReLU. A função `relu` recebe um valor e retorna o valor máximo entre 0 e o valor de entrada. Se o valor for menor que 0, a função retorna 0. Caso contrário, retorna o próprio valor.

In [6]:
def relu(self: Valor) -> Valor:
    """
    Aplica a função ReLU (Rectified Linear Unit) ao valor. A função ReLU retorna o valor se ele for positivo, caso contrário retorna 0. Realiza a operação: max(0, self).
    
    Args:
        Valor (self): valor a ser processado.
        
    Returns:
        Valor: novo objeto representando o resultado da função ReLU.
    """
    x = self
    data = x.data if x.data > 0 else 0.0
    resultado = Valor(data, (x,), "ReLU")

    def propagar_relu():
        x.grad += resultado.grad * (1.0 if x.data > 0 else 0.0)
    
    resultado.propagar = propagar_relu
    return resultado

### 2️⃣ Tanh (Tangente Hiperbólica)

A função tanh produz valores no intervalo de -1 a +1. Isso significa que ele pode lidar com valores negativos de forma mais eficaz do que a função sigmóide, que tem um intervalo de 0 a 1. [3]

$$
\tanh(x) = \frac{(e^x - e^{-x})}{(e^x + e^{-x})}
$$

As principais características que podemos destacar da função tanh são:

- Saída entre (-1, 1), centrada em zero.
- Derivada maior que a da sigmoide, o que acelera o aprendizado.

A saída entre (-1, 1) significa que a função tanh pode lidar com valores negativos de forma mais eficaz do que a função sigmoide, que tem um intervalo de 0 a 1. Isso pode ser útil em redes neurais profundas, onde a normalização dos dados pode ser importante. A derivada da função tanh é maior do que a da sigmoide, o que significa que os gradientes são mais fortes e podem acelerar o aprendizado. Isso pode levar a uma convergência mais rápida durante o treinamento.

<center>
<img src="images/Tanh.jpg" alt="Tanh Function" width="300"/>
<p>Legenda 3: Esta já é a função tanh. Ela é linear para valores próximos de zero e não linear para valores extremos.</p>
</center>

Comparando com a função sigmoide, a tanh é mais adequada para entradas centradas em zero. Isso significa que a função tanh pode lidar melhor com dados que têm uma média próxima de zero, o que pode ser útil em muitas aplicações de aprendizado de máquina. Entretanto, a tanh ainda sofre com o problema do desvanecimento de gradiente em extremos, onde os gradientes se tornam muito pequenos e dificultam o aprendizado.

Bom, agora vamos criar a função em Python para a tanh. A função `tanh` recebe um valor e retorna o valor da tangente hiperbólica do valor de entrada. Se o valor for menor que 0, a função retorna -1. Caso contrário, retorna o próprio valor.

In [7]:
def tanh(self: Valor) -> Valor:
    """
    Aplica a função tangente hiperbólica (tanh) ao valor. A função tanh retorna o valor normalizado entre -1 e 1. Realiza a operação: tanh(self).

    Args:
        Valor (self): valor a ser processado.

    Returns:
        Valor: novo objeto representando o resultado da função tanh.
    """
    x = self
    e_pos = (x * 2).exp()
    t = (e_pos - 1) / (e_pos + 1)
    resultado = Valor(t.data, (x,), "tanh")

    def propagar_tanh():
        x.grad += resultado.grad * (1 - t.data**2)

    resultado.propagar = propagar_tanh
    return resultado

### 3️⃣ Leaky ReLU

Leaky ReLU é uma versão melhorada da função ReLU para resolver o problema de "morte" da ReLU, pois tem uma pequena inclinação positiva na área negativa.

$$
\text{Leaky ReLU}(x) = 
\begin{cases}
x & \text{se } x > 0 \\
\alpha x & \text{caso contrário}
\end{cases}
\quad \text{(tipicamente } \alpha = 0.01\text{)}
$$

As principais características que podemos destacar da função Leaky ReLU são:

- Variante do ReLU que permite gradiente pequeno quando $ x < 0 $.
- Reduz o risco de "neurônios mortos".

A possibilidade de permitir um pequeno gradiente negativo quando $ x < 0 $ acontece devido à inclinação pequena, o que significa que mesmo quando a entrada é negativa, a função ainda tem um pequeno gradiente positivo. Isso ajuda a evitar o problema de "neurônios mortos", onde os neurônios não se ativam e não contribuem para o aprendizado. A Leaky ReLU é uma função de ativação mais robusta do que a ReLU padrão, pois pode lidar melhor com entradas negativas e ainda mantém as vantagens da ReLU em termos de eficiência computacional.

<center>
<img src="images/Leaky ReLU.jpg" alt="Tanh Function" width="300"/>
<p>Legenda 4: Por fim, temos a Leaky ReLU. Ela é linear para valores positivos e tem uma pequena inclinação negativa para valores negativos.</p>
</center>

Realizando uma comparação com a função sigmoide, a Leaky ReLU é mais eficiente em termos computacionais, pois não envolve operações exponenciais ou trigonométricas. Isso a torna adequada para redes neurais profundas, onde a eficiência computacional é crucial. Além disso, a Leaky ReLU não sofre com o problema do desvanecimento do gradiente da mesma forma que a sigmoide e a tanh, permitindo que os gradientes fluam mais facilmente através das camadas da rede.

Para terminar, vamos criar a função em Python para a Leaky ReLU. A função `leaky_relu` recebe um valor e retorna o valor máximo entre 0 e o valor de entrada. Se o valor for menor que 0, a função retorna 0.01 vezes o valor de entrada. Caso contrário, retorna o próprio valor.

In [8]:
def leaky_relu(self, alpha=0.01):
    """
    Aplica a função Leaky ReLU ao valor. A função Leaky ReLU retorna o valor se ele for positivo, caso contrário retorna alpha * valor. Realiza a operação: max(0, self) + alpha * min(0, self).

    Args:
        Valor (self): valor a ser processado.
        alpha (float): coeficiente de inclinação para valores negativos.

    Returns:
        Valor: novo objeto representando o resultado da função Leaky ReLU.
    """
    x = self
    data = x.data if x.data > 0 else alpha * x.data
    resultado = Valor(data, (x,), "LeakyReLU")

    def propagar_leaky():
        x.grad += resultado.grad * (1.0 if x.data > 0 else alpha)

    resultado.propagar = propagar_leaky
    return resultado

### 🧪 Testando as funções de ativação

Primeiro, para testarmos essas funções de ativação, precisamos instanciar a classe `Valor` e criar um valor.

In [9]:
Valor.relu = relu
Valor.tanh = tanh
Valor.leaky_relu = leaky_relu

Agora que definimos na classe `Valor` as funções de ativação que definimos anteriormente, vamos utilizá-las com o mesmo exemplo que o professor usou na aula de MLP.

In [10]:
def treinar_mlp(
    x,
    y_true,
    arquitetura,
    ativacao="sigmoid",
    num_epocas=200,
    taxa_de_aprendizado=0.01,
    verbose_interval=50,
):
    n_in = len(x[0])
    rede = MLP(n_in, arquitetura, ativacao=ativacao)

    loss_hist = []

    for ep in range(num_epocas):
        y_pred = [rede([Valor(v) for v in amostra]) for amostra in x]

        erros = [(yp - yt) ** 2 for yt, yp in zip(y_true, y_pred)]
        loss = sum(erros) / len(erros)

        for p in rede.parametros():
            p.grad = 0

        loss.propagar_tudo()

        for p in rede.parametros():
            p.data -= taxa_de_aprendizado * p.grad

        loss_hist.append(loss.data)
        if verbose_interval and ep % verbose_interval == 0:
            print(f"[{ativacao}] época {ep:4d}  loss={loss.data:.6f}")

    preds = [rede([Valor(v) for v in amostra]).data for amostra in x]
    return rede, preds, loss_hist

In [11]:
x = [
    [2.0,  3.0, -1.0],
    [3.0, -1.0,  0.5],
    [0.5,  1.0,  1.0],
    [1.0,  1.0, -1.0],
]
y_true = [1, 0, 0.2, 0.5]

arquitetura = [3, 2, 1]

configs = {
    "sigmoid":     dict(num_epocas=1000, taxa_de_aprendizado=0.5),
    "tanh":        dict(num_epocas=2000, taxa_de_aprendizado=0.05),
    "relu":        dict(num_epocas=2000, taxa_de_aprendizado=0.05),
    "leaky_relu":  dict(num_epocas=2000, taxa_de_aprendizado=0.01),
}

resultados = {}
for ativ, cfg in configs.items():
    print(f"\n=== {ativ.upper()} ===")
    _, preds, hist = treinar_mlp(
        x, y_true,
        arquitetura=arquitetura,
        ativacao=ativ,
        **cfg,
        verbose_interval=cfg["num_epocas"]//5
    )
    resultados[ativ] = preds

print("\nPredições finais:")
for ativ, preds in resultados.items():
    print(f"{ativ:>10}: {['%.4f' % p for p in preds]}")



=== SIGMOID ===
[sigmoid] época    0  loss=0.176704
[sigmoid] época  200  loss=0.132572
[sigmoid] época  400  loss=0.062238
[sigmoid] época  600  loss=0.017479
[sigmoid] época  800  loss=0.007029

=== TANH ===
[tanh] época    0  loss=0.265223
[tanh] época  400  loss=0.010209
[tanh] época  800  loss=0.002614
[tanh] época 1200  loss=0.001254
[tanh] época 1600  loss=0.000788

=== RELU ===
[relu] época    0  loss=0.322500
[relu] época  400  loss=0.322500
[relu] época  800  loss=0.322500
[relu] época 1200  loss=0.322500
[relu] época 1600  loss=0.322500

=== LEAKY_RELU ===
[leaky_relu] época    0  loss=3.551047
[leaky_relu] época  400  loss=0.006452
[leaky_relu] época  800  loss=0.000356
[leaky_relu] época 1200  loss=0.000095
[leaky_relu] época 1600  loss=0.000027

Predições finais:
   sigmoid: ['0.9021', '0.0780', '0.2003', '0.5089']
      tanh: ['0.9526', '-0.0004', '0.2014', '0.5027']
      relu: ['0.0000', '0.0000', '0.0000', '0.0000']
leaky_relu: ['0.9984', '-0.0010', '0.1981', '0.5048

| Ativação                  | Faixa de saída         | Gradiente típico                                 | Resultado observado                                                            | Motivo principal                                                                                                                                                                                                                    |
| ------------------------- | ---------------------- | ------------------------------------------------ | ------------------------------------------------------------------------------ | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Sigmoide**              | 0 – 1                  | gradiente moderado em torno de 0                 | *Loss* caiu de 0.14 → 0.005; predições ≈ \[0.907, 0.046, 0.205, 0.516]         | Faixa da sigmoide coincide exatamente com os rótulos‑alvo ⇒ rede só precisou aprender escala fina; taxa de aprendizado alta (0 .5) ajudou a convergir rápido.                                                                                        |
| **tanh**                  | –1 – 1 (centrada em 0) | maior que sigmoide perto da origem; satura em ±1 | *Loss* de 0.21 → 0.001; predições ≈ \[0.950, –0.004, 0.203, 0.505]             | Como a tanh devolve valores negativos, a rede acabou empurrando a soma final ligeiramente positiva para três amostras e ≈ 0 para a segunda; precisou de mais épocas (e taxa de aprendizado 0.05) mas convergiu bem.                                  |
| **ReLU**                  | 0 – ∞ (corta < 0)      | 1 quando soma > 0; **0** quando soma ≤ 0         | *Loss* permaneceu em 0.3225; saída 0 para todas as amostras                    | Todas as somas ponderadas do neurônio de saída começaram negativas → gradiente 0 → pesos/vieses dessa camada nunca mudaram (“neurônio morto”).                                                                                      |
| **Leaky ReLU** (α = 0.01) | αx – ∞                 | α (0.01) quando soma ≤ 0; 1 quando > 0           | *Loss* despencou de 0.325 → 0.010; predições ≈ \[0.999, –0.004, –0.000, 0.501] | O gradiente pequeno (0.01) salvou o fluxo de erro, mas a saída negativa persiste para as amostras onde a soma final ficou ≤ 0; sem uma função de saída que comprima/shift, a rede não “sabe” que valores negativos são inadequados. |

## 📖 Referências

[1] https://www.v7labs.com/blog/neural-networks-activation-functions

[2] https://en.wikipedia.org/wiki/Rectifier_(neural_networks)

[3] https://www.datacamp.com/tutorial/introduction-to-activation-functions-in-neural-networks